### Paqueterías

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

pd.set_option('display.float_format', '{:,.4f}'.format)

# ***Tarea 2. Equities Risk***
### Luis Eduardo Jiménez del Muro - 12/02/2025
---

## **Ejercicio 1. Función VaR y ES (40 puntos)**

### Función que calcula el VaR

In [2]:
def var_stocks(data, positions, nc, long):
    rend = data.pct_change().dropna()
    port_value = (data.iloc[-1] * positions).sum()
    w = (data*positions).iloc[-1] / port_value
    port_rend = np.dot(w, rend.T)
    
    if long == True:
        var_p = np.percentile(port_rend, 100-nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
    else:
        var_p = np.percentile(port_rend, nc)
        var_m = (data * positions).sum(axis=1).iloc[-1] * (var_p)
        es_p = port_rend[port_rend < var_p].mean()
        es_m = (data * positions).sum(axis=1).iloc[-1] * (es_p)
        
    resultados = pd.DataFrame({
        '':['%', '$'],
        'VaR':[var_p, var_m],
        'C-VaR':[es_p, es_m]
        })
    return resultados

### Cálculo del VaR y C-VaR

In [3]:
tickers = ["AAPL", "TSLA", "AMD", "LMT", "JPM"]
positions = [2193, 1211, 3221, 761, 1231]
nc= 95
long = True

data=yf.download(tickers, start="2020-01-01", end="2023-01-01")["Adj Close"][tickers]

var_stocks(data, positions, nc, long)

[*********************100%%**********************]  5 of 5 completed


,,VaR,C-VaR
0,%,-0.0288,-0.0463
1,$,"-33,073.9077","-53,131.4658"


## **Ejercicio 2. Rebalanceo de Portafolio (40 puntos)**

### Optimización

+ Se creará una función de python que represente el funcional objetivo del problema de optimización, en este caso el cálculo del VaR.
+ El problema está sujeto a 3 restricciones: que la suma de los pesos sea igual a 1, los pesos no pueden ser mayores que 1 (solo posiciones largas) y que el VaR sea igual a $30,000$.

In [4]:
w = (data.iloc[-1] * positions) / (data.iloc[-1] * positions).sum() 
rend = data.pct_change().dropna()
port_value = (data.iloc[-1] * positions).sum() 

var_objective = 30000

def optimize_var(w):
    objective = lambda w: np.abs(np.percentile(w.dot(rend.T), 100 - nc)) * port_value if long else np.abs(np.percentile(w.dot(rend.T), nc)) * port_value

    constraints = [
        {'type': 'eq', 'fun':lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun':lambda w: objective(w) - var_objective}
    ]

    bounds = [(0, 1) for _ in tickers]
    x0 = np.ones(len(tickers))/len(tickers)

    resultado = minimize(objective, method="SLSQP", x0=x0, bounds=bounds, constraints=constraints)
    w_objetivo = resultado.x
    return w_objetivo

def resumen_rebalanceo(w, w_objetivo):
    resumen = pd.DataFrame({
    'Pesos Originales': w,
    'Pesos Objetivo': w_objetivo,
    'Comprar o Vender': np.floor((w_objetivo-w) * port_value / data.iloc[-1])
    })
    return resumen

w_objetivo = optimize_var(w)

resumen = resumen_rebalanceo(w, w_objetivo)
resumen

,Pesos Originales,Pesos Objetivo,Comprar o Vender
Ticker,,,
AAPL,0.2457,0.3312,763.0000
TSLA,0.1301,0.1503,188.0000
AMD,0.1820,0.0596,"-2,167.0000"
LMT,0.3064,0.4568,373.0000
JPM,0.1359,0.0022,"-1,212.0000"


### Comprobación

Para comprobarlo, simplemente hay que sumar la cantidad de acciones a comprar y vender a las posiciones anteriores y volver a correr la función del cálculo del VaR.

In [5]:
new_positions = positions + resumen['Comprar o Vender']
var_stocks(data, new_positions, nc, long)

,,VaR,C-VaR
0,%,-0.0262,-0.0445
1,$,"-29,992.3601","-51,054.4159"


+ El VaR en cash con las nuevas posiciones no es exactamente de $\$30,000$ dado que, al calcular el número de acciones a comprar y vender hubo redondeos porque a que no se pueden comprar acciones fraccionadas. 

+ Sin embargo, el VaR es menor a $\$30,000$ lo que significa que ahora es admisible para el inversionista.

## **Ejercicio 3. Investigación Conceptual: Rebalanceos (20 puntos)**